<a href="https://colab.research.google.com/github/hopesofbuzzy/URFU_adii/blob/main/%D0%9E%D0%9F%D0%94/clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Средний вариант с fuzzy-search

In [ ]:
!pip install rapidfuzz
!pip install spacy

In [ ]:
# extract_and_cluster_colab_L12.py

# Установка зависимостей (нужно запустить один раз)
# pip install sentence-transformers scikit-learn rapidfuzz numpy

import json
import os
import numpy as np
from sklearn.cluster import DBSCAN
from sentence_transformers import SentenceTransformer
from rapidfuzz import process
import re

# Пути к файлам (в Colab файлы лежат в корне по умолчанию)
MESSAGES_FILE = "messages.json"
CLUSTERS_FILE = "clusters_L12.json"

# Название модели (загрузится автоматически)
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

# Параметры
TEXT_SIMILARITY_THRESHOLD = 0.65
EPS_DBSCAN = 1 - TEXT_SIMILARITY_THRESHOLD
MIN_SAMPLES = 2
GEOCODE_SIMILARITY_THRESHOLD = 0.1

# Словарь топонимов Екатеринбурга
# Словарь топонимов Екатеринбурга
# Словарь топонимов Екатеринбурга
TOPONYMS = [
    "Верх-Исетский",
    "Железнодорожный",
    "Кировский",
    "Ленинский",
    "Октябрьский",
    "Орджоникидзевский",
    "Чкаловский",
    "Академический",
    "Вторчермет",
    "Втузгородок",
    "Горный Щит",
    "Елизаветинское",
    "ЖБИ",
    "Завокзальный",
    "Изумрудный",
    "Кольцово",
    "Комсомольский",
    "Короленковский",
    "Малый Исток",
    "Метеогорка",
    "Нижнесвердловский",
    "Новая Сортровка",
    "Новобелореченский",
    "Павелшина",
    "Парковый",
    "Пионерский",
    "Психбольница",
    "Рудничный",
    "Семь Ключей",
    "Сибирский тракт",
    "Синие Камни",
    "Сортировка",
    "Старая Сортровка",
    "Старый Октябрь",
    "Татищева",
    "Уктус",
    "УНЦ",
    "Уралмаш",
    "Центр",
    "Черкасская",
    "Шабровский",
    "Шарташ",
    "Широкая Речка",
    "Эльмаш",
    "Юго-Западный",
    "8 Марта",
    "Амундсена",
    "Авиационная",
    "Академика Павлова",
    "Бабушкина",
    "Бахчиванджи",
    "Белинского",
    "Бориса Ельцина",
    "Братиславская",
    "Бродова",
    "Бульвар Академика Семихатова",
    "Бутюлина",
    "Валдайская",
    "Верхняя Пышма",
    "Вилонова",
    "Водная",
    "Волгоградская",
    "Генеральская",
    "Героев России",
    "Глинки",
    "Гоголя",
    "Горького",
    "Декабристов",
    "Донбасская",
    "Дружининская",
    "Ереванская",
    "Жуковского",
    "Заводская",
    "Зои Космодемьянской",
    "Кирова",
    "Колмогорова",
    "Комсомольская",
    "Короленко",
    "Космонавтов",
    "Крауля",
    "Куйбышева",
    "Лермонтова",
    "Луначарского",
    "Малышева",
    "Мамина-Сибиряка",
    "Маршала Жукова",
    "Машинная",
    "Мельникайте",
    "Металлургов",
    "Мира",
    "Октябрьская",
    "Павла Шпагина",
    "Победы",
    "Пролетарская",
    "Проспект Космонавтов",
    "Радищева",
    "Репина",
    "Розы Люксембург",
    "Сакко и Ванцетти",
    "Свердлова",
    "Сибирский тракт",
    "Софьи Ковалевской",
    "Студенческая",
    "Татищева",
    "Техническая",
    "Тургенева",
    "Туполева",
    "Учительская",
    "Фрунзе",
    "Чапаева",
    "Чкалова",
    "Шарташская",
    "Шефская",
    "Шиловская",
    "Щорса",
    "Энгельса",
    "Площадь 1905 года",
    "Площадь Труда",
    "ЦУМ",
    "Геологическая",
    "ВИЗ",
    "Уралмаш",
    "Сакко",
    "Геолка",
    "1905",
    "Уралмашь",
    "Выксунский завод",
    "Монтажников",
    "Расточная",
    "Екатеринбург",
    "Расточная",
    "Монтажников",
    "УрФУ",
    "НВК",
    "Новокольцовский",
    "НВК"
]

def normalize_text(text: str) -> str:
    return re.sub(r'[^\w\s\dа-яё]', ' ', text.lower())

def extract_geocodes(text: str) -> list[str]:
    text = normalize_text(text)
    found = set()
    for word in text.split():
        match, score, _ = process.extractOne(word, TOPONYMS)
        if score >= 75:
            found.add(match)
    return list(found)

def geocode_similarity(g1: list[str], g2: list[str], threshold=GEOCODE_SIMILARITY_THRESHOLD) -> bool:
    if not g1 or not g2:
        return False
    set1 = set(g1)
    set2 = set(g2)
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    if union == 0:
        return False
    return (intersection / union) >= threshold

def cluster_messages(messages, model):
    # Добавляем geocode к каждому сообщению
    for msg in messages:
        msg["geocode"] = extract_geocodes(msg["text"])

    # Группируем сообщения по схожести geocode
    clusters_by_geo = []
    for msg in messages:
        assigned = False
        for geo_cluster in clusters_by_geo:
            if geocode_similarity(msg["geocode"], geo_cluster[0]["geocode"]):
                geo_cluster.append(msg)
                assigned = True
                break
        if not assigned:
            clusters_by_geo.append([msg])

    clusters = []
    cluster_id_counter = 1

    for geo_group in clusters_by_geo:
        if len(geo_group) < 2:
            continue

        texts = [m["text"] for m in geo_group]
        embeddings = model.encode(texts, convert_to_numpy=True)

        clustering = DBSCAN(eps=EPS_DBSCAN, min_samples=MIN_SAMPLES, metric='cosine').fit(embeddings)

        cluster_map = {}
        for i, label in enumerate(clustering.labels_):
            if label == -1:
                continue
            if label not in cluster_map:
                cluster_map[label] = []
            cluster_map[label].append(geo_group[i])

        for cluster_label, cluster_msgs in cluster_map.items():
            if len(cluster_msgs) < 2:
                continue

            primary_geo = cluster_msgs[0]["geocode"]
            core_emb = np.mean(model.encode([m["text"] for m in cluster_msgs], convert_to_numpy=True), axis=0)
            cluster = {
                "cluster_id": f"cl_{cluster_id_counter:03d}",
                "geocode": primary_geo,
                "message_count": len(cluster_msgs),
                "first_seen": min(m["date"] for m in cluster_msgs),
                "last_seen": max(m["date"] for m in cluster_msgs),
                "core_embedding": core_emb.tolist(),
                "examples": cluster_msgs
            }
            clusters.append(cluster)
            cluster_id_counter += 1

    return clusters

def main():
    print("Загрузка модели L12...")
    model = SentenceTransformer(MODEL_NAME, device='cpu')  # Colab может использовать GPU, если доступно

    print("Загрузка сообщений...")
    with open(MESSAGES_FILE, "r", encoding="utf-8") as f:
        messages = json.load(f)

    print(f"Обнаружено {len(messages)} сообщений.")

    print("Извлечение геокодов и кластеризация...")
    clusters = cluster_messages(messages, model)

    print(f"Создано {len(clusters)} кластеров.")

    print("Сохранение кластеров...")
    with open(CLUSTERS_FILE, "w", encoding="utf-8") as f:
        json.dump({"clusters": clusters}, f, ensure_ascii=False, indent=2)

from natasha import Doc, MorphVocab, NewsEmbedding, NewsNERTagger, NewsMorphTagger
from razdel import sentenize, tokenize

def extract_geo_with_natasha(text: str) -> list[str]:
    doc = Doc(text)
    embedding = NewsEmbedding()
    morph_tagger = NewsMorphTagger(embedding)
    ner_tagger = NewsNERTagger(embedding)

    doc.tag_morph(morph_tagger)
    doc.tag_ner(ner_tagger)

    geos = []
    for span in doc.spans:
        if span.type == 'LOC':  # или 'GPE' — Geopolitical Entity
            geos.append(span.text)
    return geos

def extract_geocodes_enhanced(text: str) -> list[str]:
    found = set()

    # 1. Fuzzy-поиск по словарю
    text_norm = normalize_text(text)
    for word in text_norm.split():
        match, score, _ = process.extractOne(word, TOPONYMS)
        if score >= 85:
            found.add(match)

    # 2. Извлечение через NER (например, natasha)
    ner_geos = extract_geo_with_natasha(text)
    for geo in ner_geos:
        # Сопоставляем с каноническим словарём
        match, score, _ = process.extractOne(geo, TOPONYMS)
        if score >= 80:
            found.add(match)

    # 3. Регулярки для паттернов
    patterns = [
        r'на\s+([а-яё]+(?:\s+[а-яё]+)?)',  # "на Ленина"
        r'улица\s+([а-яё]+(?:\s+[а-яё]+)?)',  # "улица Ленина"
        r'ул\.\s+([а-яё]+(?:\s+[а-яё]+)?)',  # "ул. Ленина"
    ]
    for pattern in patterns:
        matches = re.findall(pattern, text, re.IGNORECASE)
        for m in matches:
            match, score, _ = process.extractOne(m, TOPONYMS)
            if score >= 85:
                found.add(match)

    return list(found)

    print(f"✅ Обработка завершена. Результат в {CLUSTERS_FILE}")

if __name__ == "__main__":
  extract_geocodes_enhanced("В Кольцово объяснили, почему чемоданы пассажиров валялись на снегу.Как казалось, одна из телег наехала на снежный накат и выронила несколько чемоданов. Водитель автопоезда заметил падение багажа и вернул груз обратно.Отметим, что на взлетно-посадочную полосу багаж, ко")
    # main()

TypeError: 'NoneType' object is not iterable

# Отличный вариант с spacy


In [ ]:
!pip install sentence-transformers scikit-learn rapidfuzz numpy spacy
!python -m spacy download ru_core_news_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 106.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# extract_and_cluster_colab_spacy.py

# Установка зависимостей (нужно запустить один раз)
# pip install sentence-transformers scikit-learn rapidfuzz numpy spacy
# python -m spacy download ru_core_news_sm

import json
import os
import numpy as np
from sklearn.cluster import DBSCAN
from sentence_transformers import SentenceTransformer
from rapidfuzz import process
import re

# Пути к файлам
MESSAGES_FILE = "messages.json"
CLUSTERS_FILE = "clusters_spacy.json"

# Название модели
MODEL_NAME = "paraphrase-multilingual-MiniLM-L12-v2"

# Параметры
TEXT_SIMILARITY_THRESHOLD = 0.55
EPS_DBSCAN = 1 - TEXT_SIMILARITY_THRESHOLD
MIN_SAMPLES = 2
GEOCODE_SIMILARITY_THRESHOLD = 0.5

# Словарь топонимов Екатеринбурга
TOPONYMS = [
    "Верх-Исетский",
    "Железнодорожный",
    "Кировский",
    "Ленинский",
    "Октябрьский",
    "Орджоникидзевский",
    "Чкаловский",
    "Академический",
    "Вторчермет",
    "Втузгородок",
    "Горный Щит",
    "Елизаветинское",
    "ЖБИ",
    "Завокзальный",
    "Изумрудный",
    "Кольцово",
    "Комсомольский",
    "Короленковский",
    "Малый Исток",
    "Метеогорка",
    "Нижнесвердловский",
    "Новая Сортровка",
    "Новобелореченский",
    "Павелшина",
    "Парковый",
    "Пионерский",
    "Психбольница",
    "Рудничный",
    "Семь Ключей",
    "Сибирский тракт",
    "Синие Камни",
    "Сортировка",
    "Старая Сортровка",
    "Старый Октябрь",
    "Татищева",
    "Уктус",
    "УНЦ",
    "Уралмаш",
    "Центр",
    "Черкасская",
    "Шабровский",
    "Шарташ",
    "Широкая Речка",
    "Эльмаш",
    "Юго-Западный",
    "8 Марта",
    "Амундсена",
    "Авиационная",
    "Академика Павлова",
    "Бабушкина",
    "Бахчиванджи",
    "Белинского",
    "Бориса Ельцина",
    "Братиславская",
    "Бродова",
    "Бульвар Академика Семихатова",
    "Бутюлина",
    "Валдайская",
    "Верхняя Пышма",
    "Вилонова",
    "Водная",
    "Волгоградская",
    "Генеральская",
    "Героев России",
    "Глинки",
    "Гоголя",
    "Горького",
    "Декабристов",
    "Донбасская",
    "Дружининская",
    "Ереванская",
    "Жуковского",
    "Заводская",
    "Зои Космодемьянской",
    "Кирова",
    "Колмогорова",
    "Комсомольская",
    "Короленко",
    "Космонавтов",
    "Крауля",
    "Куйбышева",
    "Лермонтова",
    "Луначарского",
    "Малышева",
    "Мамина-Сибиряка",
    "Маршала Жукова",
    "Машинная",
    "Мельникайте",
    "Металлургов",
    "Мира",
    "Октябрьская",
    "Павла Шпагина",
    "Победы",
    "Пролетарская",
    "Проспект Космонавтов",
    "Радищева",
    "Репина",
    "Розы Люксембург",
    "Сакко и Ванцетти",
    "Свердлова",
    "Сибирский тракт",
    "Софьи Ковалевской",
    "Студенческая",
    "Татищева",
    "Техническая",
    "Тургенева",
    "Туполева",
    "Учительская",
    "Фрунзе",
    "Чапаева",
    "Чкалова",
    "Шарташская",
    "Шефская",
    "Шиловская",
    "Щорса",
    "Энгельса",
    "Площадь 1905 года",
    "Площадь Труда",
    "ЦУМ",
    "Геологическая",
    "ВИЗ",
    "Уралмаш",
    "Сакко",
    "Геолка",
    "1905",
    "Уралмашь",
    "Выксунский завод",
    "Ленина",
    "Екатеринбург",
    "Расточная",
    "Монтажников",
    "УрФУ",
    "НВК",
    "Новокольцовский",
    "Кирова",
    "Лицей",
    "Детский",
    "Детсад",
    "Школа",
    "Больница",
    "Университет"
]

def normalize_text(text: str) -> str:
    return re.sub(r'[^\w\s\dа-яё]', ' ', text.lower())

def extract_geo_with_spacy(text: str) -> list[str]:
    try:
        import spacy

        # Проверим, не пустой ли текст
        if not text or len(text.strip()) < 5:
            return []

        nlp = spacy.load("ru_core_news_sm")
        doc = nlp(text)

        geos = []
        for ent in doc.ents:
            if ent.label_ in ["LOC", "GPE", "FAC"]:  # Location, Geopolitical Entity, Facility
                geos.append(ent.text)
        return geos
    except Exception as e:
        print(f"Ошибка при извлечении геокода через spacy: {e}")
        return []

def extract_geocodes_enhanced(text: str) -> list[str]:
    found = set()

    # # 1. Fuzzy-поиск по словарю
    # text_norm = normalize_text(text)
    # for word in text_norm.split():
    #     match, score, _ = process.extractOne(word, TOPONYMS)
    #     if score >= 85:
    #         found.add(match)

    # 2. Извлечение через NER (spacy)
    ner_geos = extract_geo_with_spacy(text)
    for geo in ner_geos:
        # Сопоставляем с каноническим словарём
        match, score, _ = process.extractOne(geo, TOPONYMS)
        if score >= 70:
          found.add(geo)

    # # 3. Регулярки для паттернов
    # patterns = [
    #     r'на\s+([а-яё]+(?:\s+[а-яё]+)?)',  # "на Ленина"
    #     r'улица\s+([а-яё]+(?:\s+[а-яё]+)?)',  # "улица Ленина"
    #     r'ул\.\s+([а-яё]+(?:\s+[а-яё]+)?)',  # "ул. Ленина"
    # ]
    # for pattern in patterns:
    #     matches = re.findall(pattern, text, re.IGNORECASE)
    #     for m in matches:
    #         match, score, _ = process.extractOne(m, TOPONYMS)
    #         if score >= 85:
    #             found.add(match)

    return list(found)

def geocode_similarity(g1: list[str], g2: list[str], threshold=GEOCODE_SIMILARITY_THRESHOLD) -> bool:
    if not g1 or not g2:
        return False
    set1 = set(g1)
    set2 = set(g2)
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    if union == 0:
        return False
    return (intersection / union) >= threshold

def cluster_messages(messages, model):
    # Добавляем geocode к каждому сообщению
    for msg in messages:
        msg["geocode"] = extract_geocodes_enhanced(msg["text"])

    # Группируем сообщения по схожести geocode
    clusters_by_geo = []
    for msg in messages:
        assigned = False
        for geo_cluster in clusters_by_geo:
            if geocode_similarity(msg["geocode"], geo_cluster[0]["geocode"]):
                geo_cluster.append(msg)
                assigned = True
                break
        if not assigned:
            clusters_by_geo.append([msg])

    clusters = []
    cluster_id_counter = 1

    for geo_group in clusters_by_geo:
        if len(geo_group) < 2:
            continue

        texts = [m["text"] for m in geo_group]
        embeddings = model.encode(texts, convert_to_numpy=True)

        clustering = DBSCAN(eps=EPS_DBSCAN, min_samples=MIN_SAMPLES, metric='cosine').fit(embeddings)

        cluster_map = {}
        for i, label in enumerate(clustering.labels_):
            if label == -1:
                continue
            if label not in cluster_map:
                cluster_map[label] = []
            cluster_map[label].append(geo_group[i])

        for cluster_label, cluster_msgs in cluster_map.items():
            if len(cluster_msgs) < 2:
                continue

            primary_geo = cluster_msgs[0]["geocode"]
            core_emb = np.mean(model.encode([m["text"] for m in cluster_msgs], convert_to_numpy=True), axis=0)
            cluster = {
                "cluster_id": f"cl_{cluster_id_counter:03d}",
                "geocode": primary_geo,
                "message_count": len(cluster_msgs),
                "first_seen": min(m["date"] for m in cluster_msgs),
                "last_seen": max(m["date"] for m in cluster_msgs),
                "core_embedding": core_emb.tolist(),
                "examples": cluster_msgs
            }
            clusters.append(cluster)
            cluster_id_counter += 1

    return clusters

def main():
    print("Загрузка модели L12...")
    model = SentenceTransformer(MODEL_NAME, device='cpu')

    print("Загрузка сообщений...")
    with open(MESSAGES_FILE, "r", encoding="utf-8") as f:
        messages = json.load(f)

    print(f"Обнаружено {len(messages)} сообщений.")

    print("Извлечение геокодов и кластеризация...")
    clusters = cluster_messages(messages, model)

    print(f"Создано {len(clusters)} кластеров.")

    print("Сохранение кластеров...")
    with open(CLUSTERS_FILE, "w", encoding="utf-8") as f:
        json.dump({"clusters": clusters}, f, ensure_ascii=False, indent=2)

    print(f"✅ Обработка завершена. Результат в {CLUSTERS_FILE}")

if __name__ == "__main__":
    print(extract_geo_with_spacy("На Ленина пробки"))
    main()

[]
Загрузка модели L12...
Загрузка сообщений...
Обнаружено 72 сообщений.
Извлечение геокодов и кластеризация...


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-2684141772.py", line 320, in <cell line: 0>
    main()
  File "/tmp/ipython-input-2684141772.py", line 308, in main
    clusters = cluster_messages(messages, model)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-2684141772.py", line 243, in cluster_messages
    msg["geocode"] = extract_geocodes_enhanced(msg["text"])
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-2684141772.py", line 207, in extract_geocodes_enhanced
    ner_geos = extract_geo_with_spacy(text)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-2684141772.py", line 184, in extract_geo_with_spacy
    nlp = spacy.load("ru_core_news_sm")
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-pac

TypeError: object of type 'NoneType' has no len()

In [ ]:
# test_cluster_pair.py

from sentence_transformers import SentenceTransformer
import numpy as np

# Название модели
MODEL_NAME = "paraphrase-multilingual-MiniLM-L12-v2"

# Параметры
TEXT_SIMILARITY_THRESHOLD = 0.5  # Порог косинусной близости

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def test_pair(text1: str, text2: str, model, threshold: float):
    emb1 = model.encode([text1], convert_to_numpy=True)[0]
    emb2 = model.encode([text2], convert_to_numpy=True)[0]

    sim = cosine_similarity(emb1, emb2)
    print(f"Текст 1: {text1}")
    print(f"Текст 2: {text2}")
    print(f"Схожесть: {sim:.4f}")
    print(f"Порог: {threshold}")
    print(f"Результат: {'✅ Объединены' if sim >= threshold else '❌ Не объединены'}")
    print("-" * 50)

def main():
    print("Загрузка модели...")
    model = SentenceTransformer(MODEL_NAME, device='cpu')  # или 'cuda', если GPU доступен

    # Примеры текстов для тестирования
    pairs = [
        (
            "На перекрестке Монтажников и Расточная три машины не поделили дорогу.",
            "ДТП на Монтажников и Расточная. Дорогу не поделили, затор на час."
        ),
        (
            "Свалка мусора на улице Кирова. Нужно убирать.",
            "На Кирова снова свалка! Уже неделю никто не убирает."
        ),
        (
            "Яма на Ленина, 15. Машина повреждена.",
            "Пробка на Монтажников и Расточная."
        ),
        (
            "В Екатеринбурге вынесли приговор банде бывших полицейских.",
            "Суд в центре Екатеринбурга вынес приговор."
        ),
        (
            "В лицее № 110 отменили занятия в начальных классах из-за проведения внеплановой дезинфекции.",
            "Лицей на день закрыл младшие классы для санобработки помещений."
        ),
        (
            "В детском саду № 222 на неделю закрывают группу из-за ремонта санузла.",
            "В лицее № 110 отменили занятия в начальных классах из-за проведения внеплановой дезинфекции."
        ),
        (
            "Светофор не работает на перекрестке 8 Марта - Куйбышева. Водители просят быть внимательнее.",
            "В детском саду № 222 на неделю закрывают группу из-за ремонта санузла."
        ),
        (
            "В гимназии № 35 открылся новый IT-технопарк для уроков робототехники и программирования.",
            "В нашей гимназии открыйти айти-технопарк! Лютая имба!"
        ),
        (
            "В небо над городом запустят праздничный салют в честь Дня города.",
            "В городе прошел традиционный осенний легкоатлетический кросс."
        )
    ]

    print(f"Тестирование пар текстов с порогом {TEXT_SIMILARITY_THRESHOLD}...\n")

    for text1, text2 in pairs:
        test_pair(text1, text2, model, TEXT_SIMILARITY_THRESHOLD)

if __name__ == "__main__":
    main()

Загрузка модели...
Тестирование пар текстов с порогом 0.5...

Текст 1: На перекрестке Монтажников и Расточная три машины не поделили дорогу.
Текст 2: ДТП на Монтажников и Расточная. Дорогу не поделили, затор на час.
Схожесть: 0.6976
Порог: 0.5
Результат: ✅ Объединены
--------------------------------------------------
Текст 1: Свалка мусора на улице Кирова. Нужно убирать.
Текст 2: На Кирова снова свалка! Уже неделю никто не убирает.
Схожесть: 0.4935
Порог: 0.5
Результат: ❌ Не объединены
--------------------------------------------------
Текст 1: Яма на Ленина, 15. Машина повреждена.
Текст 2: Пробка на Монтажников и Расточная.
Схожесть: 0.4813
Порог: 0.5
Результат: ❌ Не объединены
--------------------------------------------------
Текст 1: В Екатеринбурге вынесли приговор банде бывших полицейских.
Текст 2: Суд в центре Екатеринбурга вынес приговор.
Схожесть: 0.6820
Порог: 0.5
Результат: ✅ Объединены
--------------------------------------------------
Текст 1: В лицее № 110 отменили заняти

# Пытаемся улучшить

In [ ]:
# extract_and_cluster_colab_spacy.py

# Установка зависимостей (нужно запустить один раз)
# pip install sentence-transformers scikit-learn rapidfuzz numpy spacy
# python -m spacy download ru_core_news_sm

import json
import os
import numpy as np
from sklearn.cluster import DBSCAN
from sentence_transformers import SentenceTransformer
from rapidfuzz import process
import re

# Пути к файлам
MESSAGES_FILE = "messages.json"
CLUSTERS_FILE = "clusters_spacy.json"

# Название модели
MODEL_NAME = "paraphrase-multilingual-MiniLM-L12-v2"

# Параметры
TEXT_SIMILARITY_THRESHOLD = 0.55
EPS_DBSCAN = 1 - TEXT_SIMILARITY_THRESHOLD
MIN_SAMPLES = 2
GEOCODE_SIMILARITY_THRESHOLD = 0.5

# Словарь топонимов Екатеринбурга
TOPONYMS = [
    "Верх-Исетский",
    "Железнодорожный",
    "Кировский",
    "Ленинский",
    "Октябрьский",
    "Орджоникидзевский",
    "Чкаловский",
    "Академический",
    "Вторчермет",
    "Втузгородок",
    "Горный Щит",
    "Елизаветинское",
    "ЖБИ",
    "Завокзальный",
    "Изумрудный",
    "Кольцово",
    "Комсомольский",
    "Короленковский",
    "Малый Исток",
    "Метеогорка",
    "Нижнесвердловский",
    "Новая Сортровка",
    "Новобелореченский",
    "Павелшина",
    "Парковый",
    "Пионерский",
    "Психбольница",
    "Рудничный",
    "Семь Ключей",
    "Сибирский тракт",
    "Синие Камни",
    "Сортировка",
    "Старая Сортровка",
    "Старый Октябрь",
    "Татищева",
    "Уктус",
    "УНЦ",
    "Уралмаш",
    "Центр",
    "Черкасская",
    "Шабровский",
    "Шарташ",
    "Широкая Речка",
    "Эльмаш",
    "Юго-Западный",
    "8 Марта",
    "Амундсена",
    "Авиационная",
    "Академика Павлова",
    "Бабушкина",
    "Бахчиванджи",
    "Белинского",
    "Бориса Ельцина",
    "Братиславская",
    "Бродова",
    "Бульвар Академика Семихатова",
    "Бутюлина",
    "Валдайская",
    "Верхняя Пышма",
    "Вилонова",
    "Водная",
    "Волгоградская",
    "Генеральская",
    "Героев России",
    "Глинки",
    "Гоголя",
    "Горького",
    "Декабристов",
    "Донбасская",
    "Дружининская",
    "Ереванская",
    "Жуковского",
    "Заводская",
    "Зои Космодемьянской",
    "Кирова",
    "Колмогорова",
    "Комсомольская",
    "Короленко",
    "Космонавтов",
    "Крауля",
    "Куйбышева",
    "Лермонтова",
    "Луначарского",
    "Малышева",
    "Мамина-Сибиряка",
    "Маршала Жукова",
    "Машинная",
    "Мельникайте",
    "Металлургов",
    "Мира",
    "Октябрьская",
    "Павла Шпагина",
    "Победы",
    "Пролетарская",
    "Проспект Космонавтов",
    "Радищева",
    "Репина",
    "Розы Люксембург",
    "Сакко и Ванцетти",
    "Свердлова",
    "Сибирский тракт",
    "Софьи Ковалевской",
    "Студенческая",
    "Татищева",
    "Техническая",
    "Тургенева",
    "Туполева",
    "Учительская",
    "Фрунзе",
    "Чапаева",
    "Чкалова",
    "Шарташская",
    "Шефская",
    "Шиловская",
    "Щорса",
    "Энгельса",
    "Площадь 1905 года",
    "Площадь Труда",
    "ЦУМ",
    "Геологическая",
    "ВИЗ",
    "Уралмаш",
    "Сакко",
    "Геолка",
    "1905",
    "Уралмашь",
    "Выксунский завод",
    "Ленина",
    "Екатеринбург",
    "Расточная",
    "Монтажников",
    "УрФУ",
    "НВК",
    "Новокольцовский",
    "Кирова",
    "Лицей",
    "Детский",
    "Детсад",
    "Школа",
    "Больница",
    "Университет"
]

def normalize_text(text: str) -> str:
    return re.sub(r'[^\w\s\dа-яё]', ' ', text.lower())

def extract_geo_with_spacy(text: str) -> list[str]:
    try:
        import spacy

        # Проверим, не пустой ли текст
        if not text or len(text.strip()) < 5:
            return []

        nlp = spacy.load("ru_core_news_sm")
        doc = nlp(text)

        geos = []
        for ent in doc.ents:
            if ent.label_ in ["LOC", "GPE", "FAC"]:  # Location, Geopolitical Entity, Facility
                geos.append(ent.text)
        return geos
    except Exception as e:
        print(f"Ошибка при извлечении геокода через spacy: {e}")
        return []

def extract_geocodes_enhanced(text: str) -> list[str]:
    found = set()

    # # 1. Fuzzy-поиск по словарю
    # text_norm = normalize_text(text)
    # for word in text_norm.split():
    #     match, score, _ = process.extractOne(word, TOPONYMS)
    #     if score >= 85:
    #         found.add(match)

    # 2. Извлечение через NER (spacy)
    ner_geos = extract_geo_with_spacy(text)
    for geo in ner_geos:
        # Сопоставляем с каноническим словарём
        match, score, _ = process.extractOne(geo, TOPONYMS)
        if score >= 70:
          found.add(geo)

    # # 3. Регулярки для паттернов
    # patterns = [
    #     r'на\s+([а-яё]+(?:\s+[а-яё]+)?)',  # "на Ленина"
    #     r'улица\s+([а-яё]+(?:\s+[а-яё]+)?)',  # "улица Ленина"
    #     r'ул\.\s+([а-яё]+(?:\s+[а-яё]+)?)',  # "ул. Ленина"
    # ]
    # for pattern in patterns:
    #     matches = re.findall(pattern, text, re.IGNORECASE)
    #     for m in matches:
    #         match, score, _ = process.extractOne(m, TOPONYMS)
    #         if score >= 85:
    #             found.add(match)

    return list(found)

def geocode_similarity(g1: list[str], g2: list[str], threshold=GEOCODE_SIMILARITY_THRESHOLD) -> bool:
    if not g1 or not g2:
        return False
    set1 = set(g1)
    set2 = set(g2)
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    if union == 0:
        return False
    return (intersection / union) >= threshold

def cluster_messages(messages, model):
    # Извлекаем геокоды
    for msg in messages:
        msg["geocode"] = extract_geocodes_enhanced(msg["text"])

    # Разделяем на два потока
    with_geo = [m for m in messages if m["geocode"]]
    without_geo = [m for m in messages if not m["geocode"]]

    clusters = []
    cluster_id_counter = 1

    # --- Часть 1: сообщения с геокодами ---
    # Группируем по гео-похожести
    geo_groups = []
    for msg in with_geo:
        assigned = False
        for group in geo_groups:
            if geocode_similarity(msg["geocode"], group[0]["geocode"]):
                group.append(msg)
                assigned = True
                break
        if not assigned:
            geo_groups.append([msg])

    # Кластеризуем каждую гео-группу
    for group in geo_groups:
        if len(group) < MIN_SAMPLES:
            continue

        # Определяем, насколько геокоды однородны
        all_geos = [g for m in group for g in m["geocode"]]
        unique_geos = set(all_geos)
        # if len(unique_geos) == 1:
        #     relaxed_threshold = max(0.3, TEXT_SIMILARITY_THRESHOLD - 0.1)
        # elif len(unique_geos) <= 2:
        #     relaxed_threshold = max(0.35, TEXT_SIMILARITY_THRESHOLD - 0.05)
        # else:
        relaxed_threshold = TEXT_SIMILARITY_THRESHOLD

        eps = 1 - relaxed_threshold
        texts = [m["text"] for m in group]
        embeddings = model.encode(texts, convert_to_numpy=True)
        clustering = DBSCAN(eps=eps, min_samples=MIN_SAMPLES, metric='cosine').fit(embeddings)

        # Собираем кластеры
        for label in set(clustering.labels_) - {-1}:
            cluster_msgs = [group[i] for i, l in enumerate(clustering.labels_) if l == label]
            if len(cluster_msgs) >= MIN_SAMPLES:
                core_emb = np.mean(model.encode([m["text"] for m in cluster_msgs], convert_to_numpy=True), axis=0)
                clusters.append({
                    "cluster_id": f"cl_{cluster_id_counter:03d}",
                    "geocode": cluster_msgs[0]["geocode"],
                    "message_count": len(cluster_msgs),
                    "first_seen": min(m["date"] for m in cluster_msgs),
                    "last_seen": max(m["date"] for m in cluster_msgs),
                    "core_embedding": core_emb.tolist(),
                    "examples": cluster_msgs
                })
                cluster_id_counter += 1

    # --- Часть 2: сообщения без геокодов ---
    if len(without_geo) >= MIN_SAMPLES:
        texts = [m["text"] for m in without_geo]
        embeddings = model.encode(texts, convert_to_numpy=True)
        # Используем исходный порог (не меняем!)
        eps = 1 - TEXT_SIMILARITY_THRESHOLD
        clustering = DBSCAN(eps=eps, min_samples=MIN_SAMPLES, metric='cosine').fit(embeddings)

        for label in set(clustering.labels_) - {-1}:
            cluster_msgs = [without_geo[i] for i, l in enumerate(clustering.labels_) if l == label]
            if len(cluster_msgs) >= MIN_SAMPLES:
                core_emb = np.mean(model.encode([m["text"] for m in cluster_msgs], convert_to_numpy=True), axis=0)
                clusters.append({
                    "cluster_id": f"cl_{cluster_id_counter:03d}",
                    "geocode": [],  # явно указываем отсутствие
                    "message_count": len(cluster_msgs),
                    "first_seen": min(m["date"] for m in cluster_msgs),
                    "last_seen": max(m["date"] for m in cluster_msgs),
                    "core_embedding": core_emb.tolist(),
                    "examples": cluster_msgs
                })
                cluster_id_counter += 1

    return clusters

def main():
    print("Загрузка модели L12...")
    model = SentenceTransformer(MODEL_NAME, device='cpu')

    print("Загрузка сообщений...")
    with open(MESSAGES_FILE, "r", encoding="utf-8") as f:
        messages = json.load(f)

    print(f"Обнаружено {len(messages)} сообщений.")

    print("Извлечение геокодов и кластеризация...")
    clusters = cluster_messages(messages, model)

    print(f"Создано {len(clusters)} кластеров.")

    print("Сохранение кластеров...")
    with open(CLUSTERS_FILE, "w", encoding="utf-8") as f:
        json.dump({"clusters": clusters}, f, ensure_ascii=False, indent=2)

    print(f"✅ Обработка завершена. Результат в {CLUSTERS_FILE}")

if __name__ == "__main__":
    print(extract_geo_with_spacy("На Ленина пробки"))
    main()

[]
Загрузка модели L12...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Загрузка сообщений...
Обнаружено 47 сообщений.
Извлечение геокодов и кластеризация...
Создано 13 кластеров.
Сохранение кластеров...
✅ Обработка завершена. Результат в clusters_spacy.json
